<a href="https://colab.research.google.com/github/rupeshpatro2001/PeopleBox-Data-Analyst-Intern/blob/main/PeopleBox_Data_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv("/content/input.csv")


In [3]:
df.head()

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date
0,1,NaN,2021-01-01,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,2021-01-01,NaN,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01
2,3,1.0,2021-01-01,2023-12-31,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01


In [4]:
# Sort the DataFrame by Employee Code and Effective Date
df['Date of Joining'] = pd.to_datetime(df['Date of Joining'])
df.sort_values(by=['Employee Code', 'Date of Joining'], inplace=True)

In [5]:
# Initialize lists to store transformed data
output_data = []

In [6]:
# Iterate over each row to transform the data
for index, row in df.iterrows():
    # Define effective date and end date for each historical record
    effective_date = row['Date of Joining']
    if index < len(df) - 1:
        end_date = df.iloc[index + 1]['Date of Joining'] - pd.Timedelta(days=1)
    else:
        end_date = pd.to_datetime('2100-01-01')  # Far-future date for the latest record

    # Determine last compensation, last pay raise date, and variable pay
    last_compensation = row['Compensation']
    last_pay_raise_date = row['Compensation 2 date']
    variable_pay = row['Compensation 1']

    # Handle missing data and date ranges appropriately
    if pd.isnull(last_pay_raise_date):
        last_pay_raise_date = df.iloc[index - 1]['Compensation 2 date']
    if pd.isnull(variable_pay):
        variable_pay = df.iloc[index - 1]['Compensation 1']

    # Append transformed data to the output list
    output_data.append([
        row['Employee Code'],
        row['Manager Employee Code'],
        last_compensation,
        row['Compensation'],
        last_pay_raise_date,
        variable_pay,
        row['Date of Joining'],  # Tenure in Org
        row['Review 2'],  # Performance Rating
        row['Engagement 2'],  # Engagement Score
        effective_date,
        end_date
    ])

    # Check for additional engagement and review data
    for i in range(1, 3):
        if not pd.isnull(row[f'Review {i} date']):
            output_data.append([
                row['Employee Code'],
                row['Manager Employee Code'],
                last_compensation,
                row['Compensation'],
                last_pay_raise_date,
                variable_pay,
                row['Date of Joining'],  # Tenure in Org
                row[f'Review {i}'],  # Performance Rating
                row[f'Engagement {i}'],  # Engagement Score
                row[f'Review {i} date'],
                end_date
            ])

In [7]:
# Create a new DataFrame with the transformed data
output_df = pd.DataFrame(output_data, columns=[
    'Employee Code', 'Manager Employee Code', 'Last Compensation', 'Compensation',
    'Last Pay Raise Date', 'Variable Pay', 'Tenure in Org', 'Performance Rating',
    'Engagement Score', 'Effective Date', 'End Date'
])

<ipython-input-7-8eacf43a1430>:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  output_df = pd.DataFrame(output_data, columns=[


In [8]:
output_df

,Employee Code,Manager Employee Code,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date
0,1,NaN,20000,20000,2023-01-01,10000.0,2021-01-01,NaN,NaN,2021-01-01,2020-12-31
1,2,1.0,20000,20000,2023-01-01,10000.0,2021-01-01,9.5,5.0,2021-01-01,2020-12-31
2,2,1.0,20000,20000,2023-01-01,10000.0,2021-01-01,9.0,4.0,2021-06-01,2020-12-31
3,2,1.0,20000,20000,2023-01-01,10000.0,2021-01-01,9.5,5.0,2022-06-01,2020-12-31
4,3,1.0,20000,20000,2023-01-01,10000.0,2021-01-01,9.5,5.0,2021-01-01,2100-01-01
5,3,1.0,20000,20000,2023-01-01,10000.0,2021-01-01,9.0,4.0,2021-06-01,2100-01-01
6,3,1.0,20000,20000,2023-01-01,10000.0,2021-01-01,9.5,5.0,2022-06-01,2100-01-01


In [9]:
output_df.to_csv('output.csv', index=False)